<h1> Dataset Construction for Current Model Validation </h1>

<h3> In this notebook, we will download a lot of images that have a crop and determine what the false positive rate for filtration is for this dataset. We want this false positive rate to be as close to zero as possible. </h3>

In [1]:
import boto3
import glob
import json
import os
import pandas as pd
from skimage.transform import resize
from skimage.io import imread, imsave
import shutil
import tqdm

from multiprocessing import Pool

<h1> Load database dump into PANDAS </h1>

In [2]:
analysis_df_path = '/root/data/alok/crop_data/data_dumps/historical_fish_detections.csv'
analysis_df = pd.read_csv(analysis_df_path)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,23,24,25,26,27,29,30,31,33,34,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h1> Modify the dataframe as necessary </h1>

In [3]:
full_image_url_prefix = 'https://s3-eu-west-1.amazonaws.com/aquabyte-images-raw'
def full_image_key_from_crop_key(crop_key):
    key_components = crop_key.replace('.jpg', '').split('/')
    full_image_f_name = '_'.join(key_components[-1].split('_')[:-4]) + '.jpg'
    full_image_key = os.path.join('/'.join(key_components[:-1]), full_image_f_name)
    return full_image_key
    

In [4]:
analysis_df['full_image_key'] = analysis_df.image_key.apply(full_image_key_from_crop_key)
mask = (analysis_df.detected_at >= '2018-12-02') & ~(analysis_df.is_bad_crop == True)

<h1> Get unique full image paths that contain fish and download locally from S3 </h1>

In [5]:
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_credentials['aws_access_key_id'],
    aws_secret_access_key=aws_credentials['aws_secret_access_key']
)


In [6]:
full_images_dir = '/root/data/alok/filtration_classifier_data/fish_present/images'
full_image_bucket = 'aquabyte-images-raw'

def download_locally(key):
    full_image_f_name = os.path.basename(key)
    full_image_f = os.path.join(full_images_dir, full_image_f_name)
    if not os.path.exists(full_image_f):
        print('Writing image to disk: {}'.format(full_image_f))
        s3_client.download_file(full_image_bucket, key, full_image_f)

In [68]:
p = Pool(20)
N = 10
tdf = analysis_df[mask].sample(N)
keys = list(set(tdf.full_image_key.tolist()))
p.map(download_locally, keys)


Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545474244601.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1544443814338.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545992017889.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545141669740.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545739941775.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545826464220.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_present/images/left_blom-kjeppevikholmen_2_1545994610894.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/fish_prese

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
corrupt_files = []
for f in os.listdir(full_images_dir):
    if not f.endswith('.jpg'):
        corrupt_files.append(f)
        
print('rm -rf {}'.format(' '.join(corrupt_files)))

<h1> Get unique full image paths that contain fish and download locally from S3 </h1>

In [7]:
frames_df_path = '/root/data/alok/filtration_classifier_data/data_dumps/full_frames.csv'
frames_df = pd.read_csv(frames_df_path)
full_image_keys = frames_df.s3_key.tolist()
zero_crop_image_keys = list(set(full_image_keys) - set(analysis_df.full_image_key.tolist()))



In [8]:
full_images_dir = '/root/data/alok/filtration_classifier_data/zero_crops/images'
p = Pool(20)
N = 5000
p.map(download_locally, zero_crop_image_keys[:N])


Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547122766590.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546868007693.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547633679946.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547116102448.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546258286089.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548061668290.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547453314223.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_b

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1544784196901.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548675631441.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547723232227.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548669692419.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546009128978.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547634094949.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548412803043.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_b

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/right_blom-kjeppevikholmen_2_1544367034853.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548404443760.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548237764946.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1544185812089.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545470009627.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545816485287.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545657820300.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547635606961.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547107523416.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545647665382.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548405116867.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/right_blom-kjeppevikholmen_2_1544709324295.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548678812454.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547025432630.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547897029084.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546677656317.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547367865803.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547901045901.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547548517465.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548078975053.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546435965391.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_b

...

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547809760567.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548424224040.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545832207182.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547293551427.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547110544164.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547808512516.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547631952631.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/right_

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548665140401.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548071224209.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545647542383.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/right_blom-kjeppevikholmen_2_1544617636927.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1545746709739.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547812360278.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1544875578016.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547886654368.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548668430413.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548148248886.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546248619045.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1544956921187.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548497585789.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548427017730.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_b

Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547994272755.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1546343664925.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1544949258577.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548069351358.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547625780141.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1548167763512.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_blom-kjeppevikholmen_2_1547544406384.jpg
Writing image to disk: /root/data/alok/filtration_classifier_data/zero_crops/images/left_b

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,